### Import

In [ ]:
import os
import sys
from dotenv import load_dotenv

load_dotenv("../.env")
code_path = os.environ.get("CODE_PATH")
doc_path = os.environ.get("DOC_PATH")
sys.path.append(code_path)

In [ ]:
from langchain_core.messages import (
    AIMessage,
    ToolMessage
)
from langchain_core.messages.tool import tool_call
from langchain_core.tools import (
    tool,
    StructuredTool,
    BaseTool
)
from langchain_core.utils.function_calling import (
    convert_to_json_schema,
    convert_to_openai_tool,
)
import json
from pydantic import (
    BaseModel,
    Field
)
from model import (
    get_fake_chat_message,
    print_dict,
)
from langchain_core.utils.function_calling import (
    convert_to_json_schema,
    convert_to_openai_tool,
    convert_to_openai_function
)

### Tool

In [ ]:
# use @tool decorator to create a tool
# include type and help string to provide more information to the LLM
@tool
def add_number_tool(x: int, y: int) -> int:
    """Add two numbers"""
    return x + y

print(add_number_tool.__class__) # StructuredTool

# run the tool
res = add_number_tool.invoke({"x": 2, "y": 3})
print(res.__class__)

print(res)

In [ ]:
# use pydantic model to provide more information about the tool to the LLM
class IncreasePricesInput(BaseModel):
    prices: list[float] = Field(description="List of prices to increase")
    increase_factor: float = Field(description="Factor by which to increase the prices")

class IncreasePricesOutput(BaseModel):
    prices: list[float] = Field(description="List of prices")

# use @tool decorator to create a tool
@tool(args_schema=IncreasePricesInput)
def increase_prices(prices: list[float], increase_factor: float) -> IncreasePricesOutput:
    """Increase a list of prices by multiplying them with an increase factor"""
    return IncreasePricesOutput(prices = [round(price * increase_factor, 2) for price in prices])

print(increase_prices.__class__) # StructuredTool

res = increase_prices.invoke({ "prices": [2.5,2.8,3.3], "increase_factor": 1.5})
print(res.__class__)
print(res.__dict__)

In [ ]:
# use pydantic model to provide more information about the tool to the LLM
class MultiplyInput(BaseModel):
    x: int = Field(description="Number")
    y: int = Field(description="Another number")

# include type and help string to provide more information to the LLM
def multiply_number(x: int, y: int) -> int:
    """Multiply 2 numbers"""
    return x * y

# create a tool
multiply_number_tool = StructuredTool.from_function(
    func=multiply_number,
    name="Multiplication",
    description="Multiply 2 numbers",
    args_schema=MultiplyInput,
    return_direct=True,
)

# run the tool
res = multiply_number_tool.invoke({"x": 2, "y": 3})
print(res.__class__)
print(res)

### Tool Call

In [ ]:
# track the tools
tools: dict[str, BaseTool] = {
    "multiply_number_tool" : multiply_number_tool,
    "increase_prices" : increase_prices,
    "add_number_tool" : add_number_tool
}

In [ ]:
# An example of model responded with tool calls
ai_msg = AIMessage(
    content = "",
    tool_calls = [
            tool_call( # create ToolCall object
                name="increase_prices", 
                args = { "prices": [2.5,2.8,3.3], "increase_factor": 1.5}, 
                id = "tool_call_id_1"),
        ]
)

model = get_fake_chat_message([ai_msg])

In [ ]:
# return response for tool calls
def run_tool(msg: AIMessage, tools: list[BaseTool]) -> list[ToolMessage]:
    tool_messages: list[ToolMessage] = []
    for tool in msg.tool_calls: # response contains tool calls
        tool_name = tool["name"]
        if tool_name in tools: # lookup tool
            res = tools[tool_name].invoke(tool["args"]) # call the tool
            tool_messages.append(ToolMessage( # return result as ToolMessage
                content=res,
                artifact={},
                tool_call_id=tool["id"],
            ))

    return tool_messages

In [ ]:
# expect a tool call response
res: AIMessage = model.invoke("")
if res.tool_calls:
    messages = run_tool(res, tools)
    for msg in messages:
        print_dict(msg.__dict__)
else:
    print(res.content)

### Output Schema

In [ ]:
# convert a Tool to schema as required by LLM
formatted_tool = convert_to_openai_tool(increase_prices)
tool_name = formatted_tool["function"]["name"]
print_dict(formatted_tool)

In [ ]:
print_dict(IncreasePricesInput.model_json_schema())

In [ ]:
print_dict(convert_to_json_schema(IncreasePricesInput))